# Jdawg

In [ ]:
//| export

import { loadNb } from "jurassic/notebooks.ts";
import { loadEnv, md } from "jurassic/utils.ts";

In [ ]:
//| export

import Anthropic from "@anthropic-ai/sdk";

export interface Message {
  role: "assistant" | "user";
  content: string;
  timestamp: Date;
}

export class Conversation {
  private messages: Message[] = [];
  private claude: Anthropic;
  private notebookPath: string;
  private system: string;
  private model: string = "claude-3-5-sonnet-20241022";

  constructor(system: string, notebookPath: string) {
    this.claude = new Anthropic({
      apiKey: Deno.env.get("ANTHROPIC_API_KEY"),
    });
    this.system = system;
    this.notebookPath = notebookPath;
  }

  private async getContext(): Promise<string> {
    const nb = await loadNb(this.notebookPath);
    const allCells = [];

    for (const cell of nb.cells) {
      if (cell.cell_type === "code") {
        allCells.push(
          [
            "```typescript",
            cell.source.join(""),
            "```",
          ].join("\n"),
        );
      } else if (cell.cell_type === "markdown") {
        allCells.push(cell.source.join(""));
      }
    }

    return allCells.join("\n\n");
  }

  private cleanAssistantMessage(message: string): string {
    if (message.startsWith("```markdown")) {
      return message.replace(/^```markdown/ig, "").replace(/```$/ig, "");
    }
    return message;
  }

  public async complete(content: string): Promise<string> {
    this.messages.push({
      role: "user",
      content,
      timestamp: new Date(),
    });

    try {
      const response = await this.claude.messages.create({
        model: this.model,
        max_tokens: 1024,
        system: [
          this.system,
          "Here is the context of the module you are currently working on and will be asked to assist with",
          await this.getContext(),
        ].join("\n\n"),
        messages: this.messages.map((msg) => ({
          role: msg.role,
          content: msg.content,
        })),
      });

      const assistantMessage = response.content[0].text;

      this.messages.push({
        role: "assistant",
        content: assistantMessage,
        timestamp: new Date(),
      });

      return this.cleanAssistantMessage(assistantMessage);
    } catch (error) {
      console.error("Error sending message to Claude:", error);
      throw error;
    }
  }
}

In [ ]:
//| export

const system =
  `You are JDawg - helpful coding assistant, part of Jurassic toolkit.

Your job is to assist with:

- coding tasks using deno 2.0 platform (typescript)
- writing documentation using markdown for the existing code

Your knowledge is current as of December 2024.

- for assertions use "jsr:@std/assert";

Respond with code snippets or markdown when needed. Only return the exact content requested. Do not add any explanations, prefacing text, or other content.
`;

let __jdawg_conversation;

export async function j(strings: TemplateStringsArray, ...expr: string[]) {
  if (!__jdawg_conversation) {
    throw new Error("JDawg is not initialized. Run `j.initialize()` first.");
  }

  const input = strings.reduce(
    (acc, str, i) => acc + str + (expr[i] || ""),
    "",
  );
  const response = await __jdawg_conversation.complete(input);

  return md(response);
}

j.initialize = (notebookPath: str) => {
  loadEnv();

  if (!Deno.env.get("ANTHROPIC_API_KEY")) {
    throw new Error("jdawg requires ANTHROPIC_API_KEY env var to be set.");
  }

  __jdawg_conversation = new Conversation(system, notebookPath);
};

[Function (anonymous)]